In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab08.ipynb")

# Lab 8: Steganography
Contributions From: Imran Khaliq-Baporia

Welcome to Lab 8! In this lab, we will be building a tool to hide secret messages inside of seemingly ordinary image files.

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Background: Image and Pixel Representation

Before we begin hiding messages inside of images, let's begin by taking a look at how pixels and images are represented in computers. 

In the digital world, one of the most common methods of encoding an encoding a pixel to be a particular color is by representing that color as a RGB triplet. A RGB triplet is a list of 3 numbers, each ranging from 0 to 255, where the first number represents the level of red, the second number represents the level of green, and the third number reprsents the level of blue in the image. The higher the number (i.e: the closer to 255), the more intense that component will be in the final color.

For example, the RGB triplet `(0, 0, 0)` represents the color black, as `0` indicates the absence of a component entirely and we are asking for the absence of all three components. In contrast, the RGB triplet `(255, 255, 255)` represents the color white, as `255` indicates the full intensity for a component, and we are asking for the full intensity of all three components. Below are some examples of other RGB triplets and the colors they represent.

<img src="https://linuxhint.com/wp-content/uploads/2022/02/image7-9.png" width="300">

So, now that we know how to represent a single pixel, how could we extropolate this to representing an entire image? Well, we can think of an image as a matrix of pixels, so we could store an image as a 2D-array of RGB triplets. Each array inside of the 2D-array represents a row of the image, and inside each of these rows are the RGB triplets for that row, stored as a list of three numbers.

As a simple example, consider thie following 3 by 3 image (consiting of 9 total pixels).

![](https://i.imgur.com/7aOJTgl.png)

This would be represented by an array that looks like this:

```
[
    [[255, 0, 0], [255, 255, 0], [255, 0, 255]],
    [[255, 128, 0], [0, 255, 255], [0, 0, 0]],
    [[128, 0, 255], [255, 255, 255], [0, 255, 0]]
]
```

Lastly, it may be helpful throughout this lab to remember that we can represent each of the numbers in the RGB triplets as binary number. Since the numbers in the triplet range anywhere between 0 and 255, there are a total of 256 different possible values. Thus, we will require at most 8 bits (or 1 byte) to represent one of these numbers. This means that each RGB triplet requires 3 bytes to represent. The above image, represented with binary numbers, would be

```
[
    [[11111111, 00000000, 00000000], [11111111, 11111111, 00000000], [11111111, 00000000, 11111111]],
    [[11111111, 10000000, 00000000], [00000000, 11111111, 11111111], [00000000, 00000000, 00000000]],
    [[10000000, 00000000, 11111111], [11111111, 11111111, 11111111], [00000000, 11111111, 00000000]]
]
```

## Part 1: Hiding a Message

In this part of the lab, we will begin by hiding a message within a "masking" image. In order to do this, we will take advantage of the limitations of the human eyes. The human eyes' ability to differentiate different colors is quite limited. So much so, in fact, that if we change the least significant bit of one of the numbers in a RGB triplet, the change is imperceptible to ordinary human eye sight. As an example, below are two colors. One is true red (the RGB triplet is `[11111111, 00000000, 00000000]`) and one is a slightly off red (the RGB triplet is `[11111110, 00000000, 00000000]`). It is nearly impossible to determine which is which using only your eyes.

<img src="https://i.imgur.com/NcWgSZg.png" width="300">

So, using this fact, we will try to secretly encode a black and white image into a "masking" image by changing the last bit of the green component of each pixel in the "masking" image.

Here is our scheme we will use to do this:

We will take in two images (represented as 2D arrays). One image is `mask` and the other is `message`. `mask` and `message` will have the same exact dimensions. `mask` will represent an innocent looking image and `message` will the secret image we want to hide. `message` will be guranteed to only contain white and black pixel values (RGB triplets of `[255, 255, 255]` and `[0, 0, 0]`). 

To produce our encoded message, we will loop through the pixels inside of `message`. If the pixel in `message` is white, then we want to set the last bit of the green value in the corresponding pixel of `mask` to 1. If the pixel in `message` is black, then we want to set the last bit of the green value in the corresponding pixel of `mask` to 0.

For this lab, we will use the following picture of Oski as our "masking" image and our secret message will be the black-and-white image saying "go bears!". For each pixel in the "go bears!" image, we will find the corresponding pixel in the Oski image, and change its green component according to whether the pixel is black or white.

![](https://i.ibb.co/5Kgxw7Z/sidebyside.png)

**Question 1:** Write a function `encode_message` that takes in two images, `original` and `message`, and that hides `message` inside of `original` using the algorithm described above.

In [ ]:
# helper functions

# Takes in an RGB triplet PIXEL and returns True iff it encodes the color white
def is_white_pixel(pixel):
    return len(pixel) == 3 and all(n == 255 for n in pixel)

# Takes in an RGB triplet PIXEL and returns True iff it encodes the color black
def is_black_pixel(pixel):
    return len(pixel) == 3 and all(n == 0 for n in pixel)

# Takes in a number NUM and returns a new version with its last bit change to NEW_LAST_BIT
def set_last_bit(num, new_last_bit):
    assert new_last_bit == 0 or new_last_bit == 1, 'new bit invalid'
    return ((num >> 1) << 1) | new_last_bit

In [ ]:
def encode_message(mask, message):
    # check that mask and message have the same dimension
    assert len(mask) == len(message), 'mismatched number of rows'
    assert list(map(len, mask)) == list(map(len, message)), 'mismatched number of columns'
    
    # check that message contains only white and black pixels
    for row in message:
        for pixel in row:
            assert is_white_pixel(pixel) or is_black_pixel(pixel), 'message must be black and white'

    # create 2D array to for outputted image, containing secret message
    output_image = []
    for r in range(len(mask)):
        output_row = []
        for c in range(len(mask[r])):
            mask_pixel = mask[r][c]
            message_pixel = message[r][c]
            new_pixel = list(mask_pixel) # make a copy of mask_pixel
            # modify new_pixel's green value according to the algorithm above
            # hint: the helper function(s) above may be helpful.
            # hint: the green component will be the second value of the pixel
            ...
            output_row.append(new_pixel)
        output_image.append(output_row)
    return output_image

In [ ]:
grader.check("q1")

Now, let's take a look at what the output image looks like if we encode our "Go bears!" message into the image of Oski!

**Note:** This may take up to a minute to execute.

In [ ]:
OSKI_IMG = 'https://pbs.twimg.com/profile_images/1276527827848818688/dfr7_4Kn_400x400.jpg'
SECRET_MESSAGE = 'https://i.ibb.co/djphCHY/gobears.png'

oski = np.asarray(Image.open(BytesIO(requests.get(OSKI_IMG).content))).tolist()
secret = np.asarray(Image.open(BytesIO(requests.get(SECRET_MESSAGE).content))).tolist()

encoded_oski = encode_message(oski, secret)

plt.imshow(encoded_oski)
plt.show()

Notice how the image looks almost exactly the same as the original image of Oski, seen below. The modifications to the last bits of the green values are completely imperceptible.

![](https://pbs.twimg.com/profile_images/1276527827848818688/dfr7_4Kn_400x400.jpg)

## Part 2: Recovering a Message

In this part, we are going to now write a function that will let us recover a message from an image that has already had a message hidden inside of it.

For this part, you will take in an image `encoded_image` that has had a secret message encoded in it using the `encode_message` function you implemented in the previous part. Your goal is to derive the original message, by examining each pixel of `encoded_image`. Remember: if the last bit of the green value is a 1, then the corresponding pixel in the secret message was white, and if the last bit of the green value is a 0, then the corresponding pixel in the secret message was black.

**Question 2:** Write a function `decode_message` that takes in an image `encoded_image`, and return the image of the secret message hidden inside of `encoded_image`.

In [ ]:
# helper function(s)

# Takes in a number NUM and returns the last bit (either 0 or 1)
def get_last_bit(num):
    return num & 1

In [ ]:
def decode_message(encoded_image):
    WHITE_PIXEL = [255, 255, 255]
    BLACK_PIXEL = [0, 0, 0]
    recovered_message = []
    for r in range(len(encoded_image)):
        recovered_row = []
        for c in range(len(encoded_image[r])):
            encoded_pixel = encoded_image[r][c]
            # determine, using the last bit of the green value, whether to
            # append a white or black pixel to the end of recovered_row
            # hint: the helper function(s) above may be helpful.
            ...
        recovered_message.append(recovered_row)
    return recovered_message

In [ ]:
grader.check("q2")

Finally, let's test our decoding method on the picture of Oski we made at the end of part 1, and make sure that we get back our original secret message containing "go bears!". If this block doesn't work, go back to Part 1 and re-run the block where you created the `encoded_oski` image.

In [ ]:
recovered_message = decode_message(encoded_oski)

plt.imshow(recovered_message)
plt.show()

If everything has gone well, we should see our original message of "go bears!". 

To wrap up Part 2 of this lab, let's decode a brand new message that we don't know the original message for. Here's a seemingly innocent looking image.

<img src="https://i.ibb.co/gM62mM4/problem.png" width="300">


However, if we run it through our decoding algorithm, we will see that we run into sort of a **problem**. ;)

In [ ]:
NORMAL_IMG = 'https://i.ibb.co/gM62mM4/problem.png'

normal_image = np.asarray(Image.open(BytesIO(requests.get(NORMAL_IMG).content))).tolist()
recovered_image = decode_message(normal_image)

plt.imshow(recovered_image)
plt.show()

## Part 3: Content Threat Removal

To conclude this lab, we will create a algorithm that tries to block/prevent this stenographic technique.

Let's say that we were trying to devise a messaging system that blocked secret messages from hiding within images, but that still allowed legit images to pass through. One way to do this is to try and run advanced steganalysis techniques on the images to detect which ones have had a message encoded within them. However, due to the difficulty of successfully implementing these techniques, it is often preferred to instead add random "noise" to the image, with the hopes of corrupting any message that may have been hiding within while still keep the image looking the same to the human eye.

For the sake of simplicity, we will implement an algorithm that adds "noise" to images by changing the last bit of all values in the RGB triplets of the image to randomly be 0 or 1.

**Question 3:** Write a function `add_noise` that takes in `image`, and return the same image with each of the least significant bits set randomly to either 0 or 1.

In [ ]:
# helper functions

# Returns either 0 or 1 at random.
def get_random_bit():
    return random.randint(0, 1)

# Takes in a number NUM and returns a new version with its last bit change to NEW_LAST_BIT
def set_last_bit(num, new_last_bit):
    assert new_last_bit == 0 or new_last_bit == 1, 'new bit invalid'
    return ((num >> 1) << 1) | new_last_bit

In [ ]:
def add_noise(image):
    noisy_image = []
    for r in range(len(image)):
        noisy_row = []
        for c in range(len(image[r])):
            pixel = image[r][c]
            # modify the red, green, and blue components of pixel
            # by changing the each of their last bits to a random bit
            # append this modified pixel to the end of noisy_row
            # hint: the helper function(s) above may be helpful.
            ...
        noisy_image.append(noisy_row)
    return noisy_image

In [ ]:
grader.check("q3")

Now, let's see what happens if we add noise to an encoded message and then try to decode it. We should it expect the decoded message to be totally garbled (looking almost like TV static). We will test this by adding noise to the image we decoded at the end of Part 2 and then trying to decode it.

For reference, here is the encoded image:

<img src="https://i.ibb.co/gM62mM4/problem.png" width="300">

In [ ]:
NORMAL_IMG = 'https://i.ibb.co/gM62mM4/problem.png'

normal_image = np.asarray(Image.open(BytesIO(requests.get(NORMAL_IMG).content))).tolist()

noisy_image = add_noise(normal_image)

plt.imshow(noisy_image)
plt.show()

Here, we see that the "noisy" version of the image looks identical to the original. This is intended, since we want legitmate images to not be visually impacted by the noise.

However, if someone were to try and decode the image, then they would run into issues.

In [ ]:
recovered_image = decode_message(normal_image)

plt.imshow(recovered_image)
plt.show()

As we can see, we no longer are getting our message back, but rather absolute garbage.

One thing to keep in mind about this method of content threat removal is that it required us to have some prior knowledge about how the message may be possibly encoded in the image. In this case, that meant that we already knew the message was being sent through the last bits of some of the values. If the sender of the message was carefuly, the could encode the message with an entirely different scheme that may be able to get past this sort of filter. This kind of represents the arms race that exists between those implementing filters and those trying to get around them.

**FOR SUBMISSION: If you run into an error (Runtime Error) at first, try running it again. There's an infrequent bug causing an error, but it goes away after re-running the submission cell.**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)